ASL Detection using CNN

Made by Aastha Kumar
LinkedIn : linkedin.com/in/aasthakumar0310
Instagram : @gxxd_vibz_only

In [2]:
!git clone https://github.com/kumarvivek9088/aslsigndataset.git

fatal: destination path 'aslsigndataset' already exists and is not an empty directory.


In [3]:
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard
import os

In [4]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
)

val_datagen = ImageDataGenerator(rescale=1./255)

batch_size = 128

train_generator = train_datagen.flow_from_directory(
    '/content/aslsigndataset/splitdataset48x48/train',
    target_size=(48, 48),
    batch_size=batch_size,
    class_mode='categorical',
    color_mode='grayscale'
)

validation_generator = val_datagen.flow_from_directory(
    '/content/aslsigndataset/splitdataset48x48/val',
    target_size=(48, 48),
    batch_size=batch_size,
    class_mode='categorical',
    color_mode='grayscale'
)

Found 1473 images belonging to 6 classes.
Found 372 images belonging to 6 classes.


In [5]:
class_names = list(train_generator.class_indices.keys())
print(class_names)

['A', 'M', 'N', 'S', 'T', 'blank']


In [6]:
model = Sequential()
# convolutional layers
model.add(Conv2D(128, kernel_size=(3,3), activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(256, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Conv2D(512, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

model.add(Flatten())
# fully connected layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
# output layer
model.add(Dense(6, activation='softmax'))

In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 46, 46, 128)       1280      
                                                                 
 max_pooling2d (MaxPooling2  (None, 23, 23, 128)       0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 23, 23, 128)       0         
                                                                 
 conv2d_1 (Conv2D)           (None, 21, 21, 256)       295168    
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 10, 10, 256)       0         
 g2D)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 10, 10, 256)       0

In [8]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = 'accuracy' )

In [9]:
!rm -rf Logs
logdir = os.path.join("Logs")
tensorboard_callback = TensorBoard(log_dir=logdir)

In [10]:
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=100,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    callbacks=[tensorboard_callback]
)

Epoch 1/100
11/11 [==============================] - 78s 7s/step - loss: 1.7924 - accuracy: 0.1792 - val_loss: 1.7891 - val_accuracy: 0.1992
Epoch 2/100
11/11 [==============================] - 77s 7s/step - loss: 1.7864 - accuracy: 0.2216 - val_loss: 1.7833 - val_accuracy: 0.2070
Epoch 3/100
11/11 [==============================] - 72s 7s/step - loss: 1.7831 - accuracy: 0.1911 - val_loss: 1.7831 - val_accuracy: 0.1797
Epoch 4/100
11/11 [==============================] - 75s 7s/step - loss: 1.7748 - accuracy: 0.2015 - val_loss: 1.7504 - val_accuracy: 0.2734
Epoch 5/100
11/11 [==============================] - 73s 7s/step - loss: 1.7391 - accuracy: 0.2312 - val_loss: 1.6379 - val_accuracy: 0.2773
Epoch 6/100
11/11 [==============================] - 73s 7s/step - loss: 1.7036 - accuracy: 0.2342 - val_loss: 1.6744 - val_accuracy: 0.2891
Epoch 7/100
11/11 [==============================] - 75s 7s/step - loss: 1.6236 - accuracy: 0.2721 - val_loss: 1.5771 - val_accuracy: 0.3086
Epoch 8/100
1

KeyboardInterrupt: 

In [13]:
%load_ext tensorboard
%tensorboard --logdir Logs

Reusing TensorBoard on port 6006 (pid 28756), started 0:00:25 ago. (Use '!kill 28756' to kill it.)

<IPython.core.display.Javascript object>

In [15]:
model_json = model.to_json()
with open("/content/drive/MyDrive/signlanguagedetectionmodel48x48.json",'w') as json_file:
    json_file.write(model_json)
model.save("/content/drive/MyDrive/signlanguagedetectionmodel48x48.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
